# Introduction

This notebook demonstrates basic usage of BioThings Explorer, an engine for autonomously querying a distributed knowledge graph. BioThings Explorer can answer two classes of queries -- "PREDICT", and "EXPLAIN".  PREDICT queries are described in [PREDICT_demo.ipynb]. Here, we describe EXPLAIN queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

EXPLAIN queries are designed to **identify plausible reasoning chains to explain the relationship between two entities**.  For example, in this notebook, we explore the question "*Why does imatinib have an effect on the treatment of asthma?*"  This query is in part motivated by the observation reported by [Cahill et al.](https://www.nejm.org/doi/full/10.1056/NEJMoa1613125).



## Step 0: Load BioThings Explorer modules

First, install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once.

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [1]:
# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## Step 1: Find representation of "asthma" and "imatinib" in BTE

In this step, BioThings Explorer translates our query strings "asthma" and "imatinib" into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [2]:
ht = Hint()
# find all potential representations of asthma
asthma_hint = ht.query("asthma")
# select the correct representation of asthma
asthma = asthma_hint['DiseaseOrPhenotypicFeature'][0]
asthma

{'mondo': 'MONDO:0004979',
 'doid': 'DOID:2841',
 'hp': 'HP:0002099',
 'umls': 'C0004096',
 'mesh': 'D001249',
 'name': 'asthma',
 'display': 'mondo(MONDO:0004979) doid(DOID:2841) hp(HP:0002099) umls(C0004096) mesh(D001249) name(asthma) ',
 'type': 'DiseaseOrPhenotypicFeature',
 'primary': {'identifier': 'mondo',
  'cls': 'DiseaseOrPhenotypicFeature',
  'value': 'MONDO:0004979'}}

In [3]:
# find all potential representations of imatinib
imatinib_hint = ht.query("imatinib")
# select the correct representation of imatinib
imatinib = imatinib_hint['ChemicalSubstance'][0]
imatinib

{'chembl': 'CHEMBL941',
 'drugbank': 'DB00619',
 'name': 'IMATINIB',
 'pubchem': 5291,
 'umls': 'C0935989',
 'mesh': 'D000068877',
 'display': 'chembl(CHEMBL941) drugbank(DB00619) name(IMATINIB) pubchem(5291) umls(C0935989) mesh(D000068877) ',
 'type': 'ChemicalSubstance',
 'primary': {'identifier': 'chembl',
  'cls': 'ChemicalSubstance',
  'value': 'CHEMBL941'}}

## Step 2: Find intermediate nodes connecting imatinib and asthma

In this section, we find all paths in the knowledge graph that connect imatinib and asthma.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

The parameters for `FindConnection` are described below:


In [4]:
help(FindConnection.__init__)

Help on function __init__ in module biothings_explorer.user_query_dispatcher:

__init__(self, input_obj, output_obj, intermediate_nodes, registry=None)
    Find relationships in the Knowledge Graph between an Input Object and an Output Object.
    
    params
    ------
    input_obj (required): must be an object returned from Hint corresponding to a specific biomedical entity.
                          Examples: 
            Hint().query("Fanconi anemia")['DiseaseOrPhenotypicFeature'][0]
            Hint().query("acetaminophen")['ChemicalSubstance'][0]
    
    output_obj (required): must EITHER be an object returned from Hint corresponding to a specific biomedical
                           entity, OR be a string or list of strings corresponding to Biolink Entity classes.
                           Examples:
            Hint().query("acetaminophen")['ChemicalSubstance'][0]
            'Gene'
            ['Gene','ChemicalSubstance']
    
    intermediate_nodes (required): the semantic

Here, we formulate a `FindConnection` query with "asthma" as the `input_ojb`, "imatinib" as the `output_obj`.  We further specify with the `intermediate_nodes` parameter that we are looking for paths joining asthma and imatinib with *one* intermediate node that is a Gene. 

In [5]:
fc = FindConnection(input_obj=asthma, output_obj=imatinib, intermediate_nodes='Gene')

We next execute the `connect` method, which performs the **query path planning** and **query path execution** process.  In short, BioThings Explorer is deconstructing the query into individual API calls, executing those API calls, then assembling the results.

A verbose log of this process is displayed below:

In [6]:
# set verbose=True will display all steps which BTE takes to find the connection
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'asthma' and 'IMATINIB'. Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene


========== QUERY #1 -- fetch all Gene entities linked to 'asthma' ==========

==== Step #1: Query path planning ====

Because asthma is of type 'DiseaseOrPhenotypicFeature', BTE will query our meta-KG for APIs that can take 'DiseaseOrPhenotypicFeature' as input

BTE found 3 apis:

API 1. biolink_disease2gene(1 API call)
API 2. mydisease.info(1 API call)
API 3. semmeddisease(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 2.1: http://mydisease.info/v1/query (POST "q=C0004096&scopes=mondo.xrefs.umls,disgenet.xrefs.umls&fields=disgenet.genes_related_to_disease&species=human&size=100")
API 3.1: http://pending.biothings.io/semmed/query (POST "q=C0004096&scopes=umls&fields=AFFECTS_reve

## Step 3: Display and Filter results
This section demonstrates post-query filtering done in Python. Later, more advanced filtering functions will be added to the **query path execution** module for interleaved filtering, thereby enabling longer query paths. More details to come...

First, all matching paths can be exported to a data frame. Let's examine a sample of those results.

In [7]:
df = fc.display_table_view()
df.sample(10)

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output,output_type
31,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"15660518,22387292,9120022,19533439,17950015",entrez:7124,TNF,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
50,asthma,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,23307299,entrez:133482,SLCO6A1,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
0,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,20528593,entrez:613,BCR,Gene,target,dgidb,dgidb_chemical2gene,None,IMATINIB,ChemicalSubstance
56,asthma,DiseaseOrPhenotypicFeature,associatedWith,disgenet,mydisease.info,None,entrez:1080,CFTR,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
10,asthma,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,20880147,umls:C1705556,None,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
36,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,22452921,umls:C1706438,None,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
14,asthma,DiseaseOrPhenotypicFeature,associatedWith,biolink,biolink_disease2gene,"27182965,29273806,24388013,29785011,20860503,3...",entrez:4088,SMAD3,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
4,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"8711779,9351591,16088626,12225254,11665340,175...",umls:C0017337,None,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
30,asthma,DiseaseOrPhenotypicFeature,associatedWith,semmed,semmeddisease,"1323298,25862459",entrez:7124,TNF,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
2,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,20528593,entrez:613,BCR,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance


While most results are based on edges from semmed, edges from DGIdb, biolink, disgenet, and drugcentral were also retrieved from their respective APIs.  

Next, let's look to see which genes are mentioned the most.

In [8]:
df.node1_name.value_counts().head(10)

CD34     4
VEGFA    3
BCR      3
TNF      3
LCK      3
TIMP1    2
CXCR4    2
PDGFA    2
GSTK1    2
SMAD3    2
Name: node1_name, dtype: int64

And let's examine the results behind that top gene in more detail:

In [9]:
df[df.node1_name=="CD34"]

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_id,node1_name,node1_type,pred2,pred2_source,pred2_api,pred2_pubmed,output,output_type
41,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"14630618,14630618",entrez:947,CD34,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
42,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"14630618,14630618",entrez:947,CD34,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
43,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"14630618,14630618",entrez:947,CD34,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance
44,asthma,DiseaseOrPhenotypicFeature,causedBy,semmed,semmeddisease,"14630618,14630618",entrez:947,CD34,Gene,molecularlyInteractsWith,semmedgene,semmedgene,None,IMATINIB,ChemicalSubstance


In [10]:
fc.display_edge_info('asthma', 'CD34')

{0: {'info': {'bts:umls': ['C1332710'],
   'bts:pubmed': ['14630618', '14630618'],
   '@type': 'Gene',
   '$input': 'bts:umls',
   '$source': 'semmed',
   '$api': 'semmeddisease'},
  'label': 'bts:causedBy',
  'source': 'semmed'},
 1: {'info': {'bts:umls': ['C1332710'],
   'bts:pubmed': ['14630618', '14630618'],
   '@type': 'Gene',
   '$input': 'bts:umls',
   '$source': 'semmed',
   '$api': 'semmeddisease'},
  'label': 'bts:causedBy',
  'source': 'semmed'}}

In [11]:
fc.display_edge_info('CD34','IMATINIB')

{0: {'info': {'@type': 'Gene',
   'bts:umls': ['C1332710'],
   '$source': 'semmedgene',
   '$api': 'semmedgene'},
  'label': 'bts:molecularlyInteractsWith',
  'source': 'semmedgene'},
 1: {'info': {'@type': 'Gene',
   'bts:umls': ['C1332710'],
   '$source': 'semmedgene',
   '$api': 'semmedgene'},
  'label': 'bts:molecularlyInteractsWith',
  'source': 'semmedgene'}}